# Imports

In [ ]:
# Global imports
import os 
import cv2
import time
import copy
import random
import pandas as pd
import numpy as np
from collections import Counter

# Torch imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from tqdm.auto import tqdm, trange
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets

torch.cuda.empty_cache()
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Sklearn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Dataset prepration


In [ ]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.CenterCrop(299),
    transforms.RandomRotation(degrees=(-15, 15)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

val_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.CenterCrop(299),
    transforms.ToTensor()
])

In [ ]:
class MyDataset(Dataset):
    def __init__(self , dataset_path , annotation_path , phase = None , transform = None):
        self.dataset_path = dataset_path
        self.annotations = pd.read_csv(annotation_path)
        self.main_data_files = os.listdir(self.dataset_path)
        sorted(self.main_data_files)
        
        self.transform = transform


    def __getitem__(self, idx):
        image_path = self.dataset_path + self.main_data_files[idx]

        # Read images:
        image = cv2.imread(image_path)
        #image = (image/255).astype("float32") # Network needs inputs in range 0-1
        
        # Read label:
        class_ = self.annotations.loc[self.annotations["image"] == self.main_data_files[idx][:-5]]['level'].values[0]
        class_ = int(class_)
        
        # Perform transformation:
        if self.transform == None:
            image = transforms.ToTensor()(image)
        if self.transform:
            image = self.transform(image)
        return image , class_

    def __len__(self):
        return len(self.main_data_files)
    
    def path_sampler(self):
        self.annotations = self.annotations.sort_values(by = ['level' , 'image'])    
        max_counts = self.annotations['level'].value_counts(ascending=True).iloc[-1]
        grouped = self.annotations.groupby('level')
        augmented_images_name = []
        for level in range(5):
            this_group = grouped.get_group(level)
            group_counts = this_group.shape[0]
            if group_counts == max_counts:
                image_names = this_group['image'].tolist()
                augmented_images_name.append(image_names)
            if group_counts < max_counts:
                image_names = this_group['image'].tolist()
                main_counts = len(image_names)
                i = 0
                while len(image_names) < max_counts:
                    image_names.append(image_names[i])
                    i += 1
                    if i == main_counts:
                        i = 0
                
                augmented_images_name.append(image_names)
        augmented_images_name = [x + '.jpeg' for sublist in augmented_images_name for x in sublist]
        return augmented_images_name

In [ ]:
'''
def get_mean_std(loader):
    # var[X] = E[X**2] - E[X]**2
    channels_sum, channels_sqrd_sum, num_batches = 0, 0, 0

    for data, _ in tqdm(loader):
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_sqrd_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = (channels_sqrd_sum / num_batches - mean ** 2) ** 0.5

    return mean, std   
    
mean , std = get_mean_std(train_loader)

print(mean , std)
'''

# Main train_loader: mean:   mean = [0.3045, 0.3578, 0.4824] , std= [0.2182, 0.2242, 0.2571]
# Main val_loader: mean:     mean = [0.3049, 0.3587, 0.4833] , std = [0.2188, 0.2247, 0.2572]

In [ ]:
'''
##########################
### MNIST DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range

BATCH_SIZE = 32 

train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

val_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break
dataloaders_dict = {'train' : train_loader , 'val' : val_loader  }

'''

# Model

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
    if feature_extracting == False:
        for param in model.parameters():
            param.requires_grad = True

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet18":
        """ Resnet18
        """
        print(model_name)
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
        
    elif model_name == "resnet34":
        """ Resnet34
        """
        model_ft = models.resnet34(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg11":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
    
    elif model_name == "vgg19":
        """ VGG11_bn
        """
        model_ft = models.vgg19(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
        
    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train' , 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            i = 0
            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                i += 1
                # forward
                with torch.set_grad_enabled(phase == 'train'):

                    if is_inception and phase == 'train':
                        # zero the parameter gradients
                        optimizer.zero_grad()
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4 * loss2

                    else:                
                        # zero the parameter gradients
                        optimizer.zero_grad()
                        outputs = model(inputs)

                        loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                 
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            print('Phase: {}, Loss: {:.4f}, Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                PATH = "best_Model_"+ str(epoch) + ".pt"
                torch.save(model.state_dict(), PATH)
            if phase == 'val':
                val_acc_history.append(epoch_acc)
        print('-' * 100)
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    return model, val_acc_history

# You need just change this part

In [ ]:
train_dataset_path = "train/"
val_dataset_path = "val/"

image_annotation = {"train/": "train_1.csv" , 
                    "val/" : "trainLabels.csv" , 
                   }

In [ ]:
# Name of model
model_name = "inception"

# Number of classes in the dataset
num_classes = 5

# Number of epochs to train for
num_epochs = 10

# Batch size
BATCH_SIZE = 32

# Learning rate of model
LEARNING_RATE = 0.001

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract =False

# Flag for using pretrain weights
use_pretrained=True

# Initialize the model for this run
model = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Send the model to GPU
model = model.to(device)

In [ ]:
# Create dataset
train_dataset = MyDataset(dataset_path = train_dataset_path , annotation_path = image_annotation[train_dataset_path] , phase = 'train' , transform = train_transforms)
val_dataset = MyDataset(dataset_path = val_dataset_path , annotation_path = image_annotation[val_dataset_path] , phase = 'val', transform = val_transforms)

train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE,  shuffle = True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE,  shuffle = True, num_workers=0)

dataloaders_dict = {'train' : train_loader , "val" : val_loader  }

In [ ]:
# Find total parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE )
criterion = nn.CrossEntropyLoss()

# Trainig model

In [ ]:
model, val_acc_history = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs = num_epochs , is_inception = (model_name=="inception"))

# Load and Test

In [ ]:
model.load_state_dict(torch.load("./best_Model_9.pt"))

In [ ]:
model = model.to(device)

In [ ]:
def test(model , loader , criterion):
    model.eval()
    test_loss = 0
    correct = 0
    All_preds = []
    All_labels = []
    with torch.no_grad():
        for images, labels in tqdm(loader):
            images = images.to(device)
            labels =  labels.to(device)
            preds = model(images)
            loss = criterion(preds , labels)
            test_loss += loss
            preds = torch.argmax(preds , dim = 1)
            All_preds.append(preds.cpu().numpy())
            All_labels.append(labels.cpu().numpy())

            correct += (preds == labels).float().sum()
        print(labels)
        print(preds)
        accuracy = correct / len(loader.dataset)    
        epoch_loss = test_loss / len(loader.dataset)
        print("   loss:%.2f" %epoch_loss.item())
        All_labels = np.concatenate( All_labels, axis=0 )
        All_preds = np.concatenate( All_preds, axis=0 )
    return All_labels ,All_preds

In [ ]:
All_labels ,All_preds = test(model , val_loader , criterion)

In [ ]:
print(confusion_matrix(All_labels ,All_preds ))
print()
print(classification_report(All_labels ,All_preds ))